I'm using GPU for this notebook, remove this code if you are not using GPU.

In [1]:
import plaidml.keras
import os
plaidml.keras.install_backend()
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

Load necessary packages and dataset

In [2]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import json
import os

# Create an empty lists to group angles and category of steer angle
categories=[]
angles=[]     

filepath = "tub280_copy\\tub_280_20-02-01(Copywithmeta)\\" # Load image from tub 
filenames = sorted(os.listdir(filepath), key=len)

for filename in filenames: 
    if filename.endswith(".json") and not filename.startswith("meta"):
            with open(filepath+filename, 'r') as read_file:
            
                json_data = json.load(read_file)
                angle_value = float(json_data["user/angle"]) 
                    
                if -1.0 <= angle_value <= -0.8:
                    categories.append("cat1")
                    angles.append(angle_value)
                 
                elif -0.8 <= angle_value <= -0.6:
                    categories.append("cat2")
                    angles.append(angle_value)
                        
                elif -0.6 <= angle_value <= -0.4:
                    categories.append("cat3")
                    angles.append(angle_value)
                                      
                elif -0.4 <= angle_value <= -0.2:
                    categories.append("cat4")
                    angles.append(angle_value)
                                      
                elif -0.2 <= angle_value <= 0.0:
                    categories.append("cat5")
                    angles.append(angle_value)
                    
                elif 0.0 <= angle_value <= 0.2:
                    categories.append("cat6")
                    angles.append(angle_value)
                
                elif 0.2 <= angle_value <= 0.4:
                    categories.append("cat7")
                    angles.append(angle_value)
                    
                elif 0.4 <= angle_value <= 0.6:
                    categories.append("cat8")
                    angles.append(angle_value)
                
                elif 0.6 <= angle_value <= 0.8:
                    categories.append("cat9")
                    angles.append(angle_value)
                    
                elif 0.8 <= angle_value <= 1.0:
                    categories.append("cat10")
                    angles.append(angle_value)
                    
# The amount of categories should correspond to amount of json files
x = len(categories)

# Store filename and category in dataframe      
# Array length of json, image and category should be the same
# *IMPORTANT* (amount in filename MUST CORRESPOND TO amount in categories)
df = pd.DataFrame({
     
    'json_filename': filenames[1:x+1],
    'image_filename': filenames[x+1:2*x+1],
    'category': categories,     
    'angle_value': angles
})

We do not need to create & compile model since we have saved the trained model already

Lets split our files into 80% train and 20% validation, 
state 0 to prevent random splitting

In [4]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=0) 

# Lets see total 'TRAIN' files and in each category
train_df = train_df.reset_index(drop=True)

In [5]:
# Lets see total 'VALIDATE' files and in each category
validate_df = validate_df.reset_index(drop=True)

In [6]:
# Get total train and validate, and batch size to fit model
total_train = train_df.shape[0]
total_validate = validate_df.shape[0] 

Use Data augmentation on the train set

Create train and validation generators

In [7]:
train_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    brightness_range = [0.8,0.8],
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Create train and valdiation generator
# From data folder, we split the data into train and validate. 
# The image in train and validate are taken from data, BUT do not contain the same images
# I checked by running 'submission.validate_df(copy)/train_df(copy)'

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    filepath, 
    x_col='image_filename',
    y_col='category',
    target_size=(120,160),
    class_mode='categorical', 
    batch_size=20,
    
)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    filepath, 
    x_col='image_filename',
    y_col='category',
    target_size=(120,160),
    class_mode='categorical',
    batch_size=20
)

Found 3933 validated image filenames belonging to 10 classes.
Found 984 validated image filenames belonging to 10 classes.


In [8]:
# No need to train model or plot graph